In [1]:
from imports import *

In [7]:
repo_url = "https://api.github.com/repos/OpportunityInsights/EconomicTracker/contents/data"

# Fonction pour télécharger les fichiers depuis un dossier GitHub, avec des filtres de formats
def download_files_from_github(repo_url, download_folder="destination",file_formats=None):
    """
    Télécharge les fichiers depuis un dossier GitHub en filtrant par extension de fichier.
    
    :param repo_url: URL API GitHub pour accéder au contenu d'un dossier
    :param file_formats: Liste ou string d'extensions de fichiers à télécharger (par exemple: ['.csv', '.json']),
                          Si None, tous les fichiers seront téléchargés.
    :param download_folder: Dossier où les fichiers seront stockés localement.
    """
    
    # Créer le dossier de destination si nécessaire
    os.makedirs(download_folder, exist_ok=True)
    
    # Fonction pour télécharger un fichier depuis une URL
    def download_file(url, file_path):
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Fichier téléchargé avec succès : {file_path}")
        else:
            print(f"Erreur lors du téléchargement de {file_path}, code : {response.status_code}")
    
    # Récupérer la liste des fichiers dans le dossier GitHub
    response = requests.get(repo_url)
    if response.status_code == 200:
        files = response.json()
        
        # Si file_formats est une string, la convertir en liste
        if isinstance(file_formats, str):
            file_formats = [file_formats]
        
        # Télécharger uniquement les fichiers filtrés et ceux qui ne sont pas déjà présents
        for file in files:
            file_name = file['name']
            file_download_url = file['download_url']
            file_extension = os.path.splitext(file_name)[1]
            
            # Vérifier si l'extension de fichier est acceptée, ou si aucun filtre n'est spécifié
            if file_formats is None or file_extension in file_formats:
                file_path = os.path.join(download_folder, file_name)
                
                # Si le fichier est déjà présent, l'ignorer
                if not os.path.exists(file_path):
                    download_file(file_download_url, file_path)
                else:
                    print(f"Le fichier {file_name} existe déjà, il est ignoré.")
    else:
        print(f"Erreur lors de la récupération des fichiers : {response.status_code}")

# avec la fonction on collecte les data
# on crée un dask dataframe à partir des données téléchargées
# et on met chaque dataframe dans un dico dont le nom de la cle fait reference au nom du fichier
#mais on check d'abord si les data sont déjà présentes

download_files_from_github(repo_url, download_folder="data", file_formats=['.csv'])

Fichier téléchargé avec succès : data\Affinity - City - Daily.csv
Fichier téléchargé avec succès : data\Affinity - County - Daily.csv
Fichier téléchargé avec succès : data\Affinity - National - Daily.csv
Fichier téléchargé avec succès : data\Affinity - State - Daily.csv
Fichier téléchargé avec succès : data\Affinity Daily Total Spending - National.csv
Fichier téléchargé avec succès : data\Affinity Income Shares - National - 2019.csv
Fichier téléchargé avec succès : data\Affinity Income Shares - National - 2020.csv
Fichier téléchargé avec succès : data\Affinity Industry Composition - National - 2020.csv
Fichier téléchargé avec succès : data\COVID - City - Daily.csv
Fichier téléchargé avec succès : data\COVID - County - Daily 2020.csv
Fichier téléchargé avec succès : data\COVID - County - Daily 2023.csv
Fichier téléchargé avec succès : data\COVID - National - Daily.csv
Fichier téléchargé avec succès : data\COVID - State - Daily.csv
Fichier téléchargé avec succès : data\Earnin - ZCTA - 20

In [9]:
files_dict = {}

for file_name in os.listdir("data"):
    if file_name.endswith(".csv"):
        #verification si le dataframe existe déjà
        if file_name not in files_dict:
            #virer l'extension pour le nom de la clé
            file_name_for_dict = file_name.split(".")[0]
            
            files_dict[file_name_for_dict] = dd.read_csv(os.path.join("data", file_name))
            
            print(f"Chargement du dataframe de {file_name_for_dict} réussi")
        else : 
            print(f"Le dataframe de {file_name_for_dict} existe déjà, il n'est pas chargé")

    if file_name.endswith(".csv.gz"):
        # verification si le dataframe existe déjà
        if file_name not in files_dict:
            #virer l'extension pour le nom de la clé
            file_name_for_dict = file_name.split(".")[0]
            files_dict[file_name_for_dict] = dd.read_csv(os.path.join("data", file_name), compression='gzip')
            print(f"Chargement du dataframe de {file_name} réussi")
        else:
            print(f"Le dataframe de {file_name} existe déjà, il n'est pas chargé")

Chargement du dataframe de Affinity - City - Daily réussi
Chargement du dataframe de Affinity - County - Daily réussi
Chargement du dataframe de Affinity - National - Daily réussi
Chargement du dataframe de Affinity - State - Daily réussi
Chargement du dataframe de Affinity Daily Total Spending - National réussi
Chargement du dataframe de Affinity Income Shares - National - 2019 réussi
Chargement du dataframe de Affinity Income Shares - National - 2020 réussi
Chargement du dataframe de Affinity Industry Composition - National - 2020 réussi
Chargement du dataframe de COVID - City - Daily réussi
Chargement du dataframe de COVID - County - Daily 2020 réussi
Chargement du dataframe de COVID - County - Daily 2023 réussi
Chargement du dataframe de COVID - National - Daily réussi
Chargement du dataframe de COVID - State - Daily réussi
Chargement du dataframe de Earnin - ZCTA - 2020 réussi
Chargement du dataframe de Employment - City - Weekly réussi
Chargement du dataframe de Employment - Coun

In [ ]:
for key, value in files_dict.items():
    display(f"\n\n{key}\n{value.info()}")


<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 24 entries, year to provisional
dtypes: int64(5), string(19)

'\n\nAffinity - City - Daily\nNone'

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 7 entries, year to provisional
dtypes: int64(5), string(2)

'\n\nAffinity - County - Daily\nNone'

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 142 entries, year to provisional
dtypes: int64(4), string(138)

...

'\n\nZearn - County - Weekly\nNone'

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 19 entries, year to break_badges_incmiddle
dtypes: float64(8), int64(3), string(8)

'\n\nZearn - National - Weekly\nNone'

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 20 entries, year to break_badges_incmiddle
dtypes: float64(4), int64(4), string(12)

'\n\nZearn - State - Weekly\nNone'

In [ ]:
"""

['Affinity - City - Daily.csv',
 'Affinity - County - Daily.csv',
 'Affinity - National - Daily.csv',
 'Affinity - State - Daily.csv',
 'Affinity Daily Total Spending - National.csv',
 'Affinity Income Shares - National - 2019.csv',
 'Affinity Income Shares - National - 2020.csv',
 'Affinity Industry Composition - National - 2020.csv',
 'COVID - City - Daily.csv',
 'COVID - County - Daily 2020.csv',
 'COVID - County - Daily 2023.csv',
 'COVID - National - Daily.csv',
 'COVID - State - Daily.csv',
 'Earnin - ZCTA - 2020.csv',
 'Employment - City - Weekly.csv',
 'Employment - County - Weekly.csv',
 'Employment - National - Weekly.csv',
 'Employment - State - Weekly.csv',
 'GeoIDs - City.csv',
 'GeoIDs - County.csv',
 'GeoIDs - State.csv',
 'Google Mobility - City - Daily.csv',
 'Google Mobility - National - Daily.csv',
 'Google Mobility - State - Daily.csv',
 'Job Postings - City - Weekly.csv',
 'Job Postings - County - Weekly.csv',
 'Job Postings - National - Weekly.csv',
 'Job Postings - State - Weekly.csv',
 'Job Postings Industry Shares - National - 2020.csv',
 'Policy Milestones - State.csv',
 'UI Claims - City - Weekly.csv',
 'UI Claims - County - Weekly.csv',
 'UI Claims - National - Weekly.csv',
 'UI Claims - State - Weekly.csv',
 'Womply - City - Weekly.csv',
 'Womply - County - Weekly.csv',
 'Womply - National - Weekly.csv',
 'Womply - State - Weekly.csv',
 'Womply - ZCTA - 2020.csv',
 'Zearn - City - Weekly.csv',
 'Zearn - County - Weekly.csv',
 'Zearn - National - Weekly.csv',
 'Zearn - State - Weekly.csv']

"""

